In [ ]:
# Question 9
import cv2
import numpy as np
import matplotlib.pyplot as plt

def complete_rice_analysis(image_path_a, image_path_b):
    """
    Q9: Complete rice grain analysis with noise removal, segmentation, and counting
    """
    # Read images
    rice_a = cv2.imread(image_path_a, cv2.IMREAD_GRAYSCALE)
    rice_b = cv2.imread(image_path_b, cv2.IMREAD_GRAYSCALE)
    
    if rice_a is None or rice_b is None:
        raise ValueError("Could not load one or both rice images")
    
    # (a) Preprocess Image 8a to remove noise (assumed Gaussian noise)
    denoised_a = cv2.fastNlMeansDenoising(rice_a, None, 10, 7, 21)
    
    # (b) Preprocess Image 8b to remove noise (assumed salt & pepper noise)
    denoised_b = cv2.medianBlur(rice_b, 5)
    
    # (c) Apply Otsu's method to segment both images
    _, binary_a = cv2.threshold(denoised_a, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    _, binary_b = cv2.threshold(denoised_b, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # (d) Apply morphological operations to remove small objects and fill holes
    kernel = np.ones((3, 3), np.uint8)
    
    # For Image A
    cleaned_a = cv2.morphologyEx(binary_a, cv2.MORPH_OPEN, kernel, iterations=2)
    cleaned_a = cv2.morphologyEx(cleaned_a, cv2.MORPH_CLOSE, kernel, iterations=2)
    
    # For Image B
    cleaned_b = cv2.morphologyEx(binary_b, cv2.MORPH_OPEN, kernel, iterations=2)
    cleaned_b = cv2.morphologyEx(cleaned_b, cv2.MORPH_CLOSE, kernel, iterations=2)
    
    # (e) Use connected components to count rice grains (using OpenCV only)
    def count_rice_grains(binary_image):
        # Invert so rice grains are white (objects)
        inverted = cv2.bitwise_not(binary_image)
        
        # Connected components analysis
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(inverted, connectivity=8)
        
        # Filter out small components (noise) and create result mask
        min_area = 50  # Minimum area for a rice grain
        rice_count = 0
        result_mask = np.zeros_like(inverted)
        
        for i in range(1, num_labels):  # Skip background (label 0)
            if stats[i, cv2.CC_STAT_AREA] >= min_area:
                rice_count += 1
                result_mask[labels == i] = 255
        
        return rice_count, result_mask
    
    count_a, rice_mask_a = count_rice_grains(cleaned_a)
    count_b, rice_mask_b = count_rice_grains(cleaned_b)
    
    return (rice_a, rice_b, denoised_a, denoised_b, 
            binary_a, binary_b, cleaned_a, cleaned_b,
            count_a, count_b, rice_mask_a, rice_mask_b)

# Main execution
def main():
    image_path_a = 'E:/UoM MSc in AI/Semester 3/IT5437 - Computer Vision/Assignment/a1images/rice_gaussian_noise.png'
    image_path_b = 'E:/UoM MSc in AI/Semester 3/IT5437 - Computer Vision/Assignment/a1images/rice_salt_pepper_noise.png'
    
    try:        
        # Complete analysis with your actual images
        (rice_a, rice_b, denoised_a, denoised_b, 
         binary_a, binary_b, cleaned_a, cleaned_b,
         count_a, count_b, rice_mask_a, rice_mask_b) = complete_rice_analysis(image_path_a, image_path_b)
        
        # Create comprehensive visualization
        plt.figure(figsize=(20, 16))
        
        # Row 1: Image A processing pipeline
        plt.subplot(4, 4, 1)
        plt.imshow(rice_a, cmap='gray')
        plt.title('(a) Image 8a - Original', fontweight='bold', fontsize=10)
        plt.axis('off')
        
        plt.subplot(4, 4, 2)
        plt.imshow(denoised_a, cmap='gray')
        plt.title('(a) Denoised (NLM)\nGaussian Noise Removal', fontweight='bold', fontsize=10)
        plt.axis('off')
        
        plt.subplot(4, 4, 3)
        plt.imshow(binary_a, cmap='gray')
        plt.title('(c) Otsu Thresholding\nBinary Segmentation', fontweight='bold', fontsize=10)
        plt.axis('off')
        
        plt.subplot(4, 4, 4)
        plt.imshow(cleaned_a, cmap='gray')
        plt.title('(d) Morphological Cleaning\nOpen + Close', fontweight='bold', fontsize=10)
        plt.axis('off')
        
        # Row 2: Image B processing pipeline
        plt.subplot(4, 4, 5)
        plt.imshow(rice_b, cmap='gray')
        plt.title('(b) Image 8b - Original', fontweight='bold', fontsize=10)
        plt.axis('off')
        
        plt.subplot(4, 4, 6)
        plt.imshow(denoised_b, cmap='gray')
        plt.title('(b) Denoised (Median)\nSalt & Pepper Removal', fontweight='bold', fontsize=10)
        plt.axis('off')
        
        plt.subplot(4, 4, 7)
        plt.imshow(binary_b, cmap='gray')
        plt.title('(c) Otsu Thresholding\nBinary Segmentation', fontweight='bold', fontsize=10)
        plt.axis('off')
        
        plt.subplot(4, 4, 8)
        plt.imshow(cleaned_b, cmap='gray')
        plt.title('(d) Morphological Cleaning\nOpen + Close', fontweight='bold', fontsize=10)
        plt.axis('off')
        
        # Row 3: Connected components results
        plt.subplot(4, 4, 9)
        plt.imshow(rice_mask_a, cmap='gray')
        plt.title(f'(e) Rice Grains Detected\nCount: {count_a}', fontweight='bold', fontsize=10)
        plt.axis('off')
        
        plt.subplot(4, 4, 10)
        plt.imshow(rice_a, cmap='gray')
        plt.imshow(rice_mask_a, cmap='jet', alpha=0.3)
        plt.title('(e) Overlay Detection\nImage 8a', fontweight='bold', fontsize=10)
        plt.axis('off')
        
        plt.subplot(4, 4, 11)
        plt.imshow(rice_mask_b, cmap='gray')
        plt.title(f'(e) Rice Grains Detected\nCount: {count_b}', fontweight='bold', fontsize=10)
        plt.axis('off')
        
        plt.subplot(4, 4, 12)
        plt.imshow(rice_b, cmap='gray')
        plt.imshow(rice_mask_b, cmap='jet', alpha=0.3)
        plt.title('(e) Overlay Detection\nImage 8b', fontweight='bold', fontsize=10)
        plt.axis('off')
        
        # Row 4: Histograms and analysis
        plt.subplot(4, 4, 13)
        plt.hist(rice_a.ravel(), bins=50, alpha=0.7, color='blue', label='Original')
        plt.hist(denoised_a.ravel(), bins=50, alpha=0.7, color='red', label='Denoised')
        plt.title('Image 8a - Histogram Comparison', fontweight='bold', fontsize=10)
        plt.xlabel('Intensity')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.subplot(4, 4, 14)
        plt.hist(rice_b.ravel(), bins=50, alpha=0.7, color='blue', label='Original')
        plt.hist(denoised_b.ravel(), bins=50, alpha=0.7, color='green', label='Denoised')
        plt.title('Image 8b - Histogram Comparison', fontweight='bold', fontsize=10)
        plt.xlabel('Intensity')
        plt.legend()
        plt.grid(True, alpha=0.3)
        
        plt.subplot(4, 4, 15)
        areas = [cv2.countNonZero(rice_mask_a), cv2.countNonZero(rice_mask_b)]
        labels = ['Image 8a', 'Image 8b']
        colors = ['lightcoral', 'lightblue']
        plt.bar(labels, areas, color=colors, alpha=0.7)
        plt.title('Total Rice Grain Area', fontweight='bold', fontsize=10)
        plt.ylabel('Pixel Area')
        plt.grid(True, alpha=0.3)
        
        plt.subplot(4, 4, 16)
        counts = [count_a, count_b]
        plt.bar(labels, counts, color=colors, alpha=0.7)
        plt.title('Rice Grain Count', fontweight='bold', fontsize=10)
        plt.ylabel('Number of Grains')
        plt.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        print(f"\n(a) Preprocess Image 8a to remove noise:")
        print("   - Applied Non-Local Means (NLM) denoising")
        print("   - Parameters: h=10, templateWindowSize=7, searchWindowSize=21")
        print("   - Chosen for Gaussian noise assumption")
        
        print(f"\n(b) Preprocess Image 8b to remove noise:")
        print("   - Applied Median Filter with 5x5 kernel")
        print("   - Chosen for salt & pepper noise assumption")
        
        print(f"\n(c) Apply Otsu's method to segment both images:")
        print("   - Used cv2.threshold with THRESH_BINARY + THRESH_OTSU")
        print("   - Automatic threshold calculation")
        
        print(f"\n(d) Apply morphological operations:")
        print("   - MORPH_OPEN (2 iterations): Removes small noise objects")
        print("   - MORPH_CLOSE (2 iterations): Fills holes in rice grains")
        print("   - 3x3 rectangular kernel used")
        
        print(f"\n(e) Use connected components to count rice grains:")
        print(f"   - Image 8a Rice Count: {count_a} grains")
        print(f"   - Image 8b Rice Count: {count_b} grains")
        print("   - Minimum area filter: 50 pixels (removes noise)")
        print("   - 8-connectivity used for accurate component detection")
        
    except Exception as e:
        print(f"Error loading images: {e}")
        print("Please check your image paths and try again.")

if __name__ == "__main__":
    main()